# Chapter 17. Future를 이용한 동시성

이 장에서는 `concurrent.futures` 라이브러리를 중점적으로 알아본다. 그리고 비동기 작업의 실행을 나타내는 객체인 `Future`의 개념에 대해 소개한다.

## 17.1 예제: 세 가지 스타일의 웹 내려받기

긴 지연 시간동안 네트워크 입출력을 효율적으로 처리하려면 동시성을 이용해야 한다. 즉, 네트워크에서 응답이 오는 동안 다른 일을 처리하는 것이 좋다. 여기서는 웹에서 20개 국가의 국기 이미지를 내려받는 프로그램을 3개 작성한다. 

1. `flags.py`는 순차적으로 실행되므로 이전 이미지를 내려받아 디스크에 저장한 후 다음번 이미지를 내려받는다. 

2. `flags_threadpool.py`는 `concurrent.futures` 패키지를 사용하여 모든 이미지를 동시에 요청한 후 도착하는 대로 파일에 저장한다.

3. `flags_asyncio.py`는 `asyncio`를 이용하여 동시에 저장한다.

여기서 동시성 스크립트들은 순차적인 스크립트보다 5배 정도 빠르다. 3번은 18장에서 소개한다.

### 17.1.1 순차 내려받기 스크립트

In [7]:
# flags.py

import os
import time
import sys

import requests
# 표준 라이브러리에 속해 있지 않으므로 한줄 띄고 임포트

# 국기 사이트가 뻗었으므로 걍 내 이미지로 하자...
POP20_CC = ('8_1 8_2 8_3 8_4 8_5 '
           '13_1 13_2 13_3 15_1').split()

#BASE_URL = 'http://flupy.org/data/flags'
BASE_URL = 'https://github.com/lih0905/Fluent_Python/raw/master/images/'

# 이미지 저장할 디렉토리 (미리 만들어놔야함)
DEST_DIR = 'downloads/'

# 이미지(바이트 시퀀스) 저장
def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)

# 이미지 이름을 인수로 받아서 URL을 만들고 이미지(이진 시퀀스)를 내려받는다.
def get_flag(cc):
    url = '{}/{cc}.PNG'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

# 문자열을 출력하고 sys.stdout.flush를 호출해서 진행 상황을 한 줄에 출력한다.
# 일반적으로 파이썬은 개행 문자를 받기 전까지 문자열을 출력하지 않으므로 
# sys.stdout.flush()를 호출해서 stdout 버퍼에 남아 있는 내용을 모두 화면에 출력하게 해야한다.
def show(text):
    print(text, end=' ')
    sys.stdout.flush()

# 동시성 버전과 다른 핵심 부분
def download_many(cc_list):
    for cc in sorted(cc_list): # 출력할 때 순서대로 나오게 만든다.
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.PNG')
    return len(cc_list)

# download_many()를 실행하는 데 걸린 시간을 기록하고 출력한다.
def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == '__main__':
    main(download_many)

13_1 13_2 13_3 15_1 8_1 8_2 8_3 8_4 8_5 
9 flags downloaded in 5.44s


### 17.1.2 concurrent.futures로 내려받기

`concurrent.futures` 패키지의 가장 큰 특징은 `ThreadPoolExecutor`와 `ProcessPoolExecutor` 클래스인데, 이 클래스들은 콜러블 객체를 서로 다른 스레드나 프로세스에서 실행할 수 있게 해주는 인터페이스를 구현한다.

In [ ]:
# flags_threadpool.py

from concurrent import futures

# flags 모듈의 함수들 재사용
from flags import save_flag, get_flag, show, main

# ThreadPoolExecutor에서 사용할 최대 스레드 수
MAX_WORKERS = 20

# 하나의 이미지를 내려받을 함수
def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.PNG')
    return cc

def download_many(cc_list):
    # 작업자 스레드 수
    workers = min(MAX_WORKERS, len(cc_list))
    # ThreadPoolExecutor 객체 생성
    with futures.ThreadPoolExecutor(workers) as executor:
        # map 메서드는 여러 스레드에 의해 download_one() 함수를 동시에 호출한다
        # 내장된 map() 함수와 비슷젠하며, 각 함수가 반환한 값을 다담은 제너레이터 반환
        res = executor.map(download_one, sorted(cc_list))
    
    # 결과 반환. 호출한 함수중 하나라도 예외를 발생시키면 여기서 에러 발생
    return len(list(res))

if __name__ == '__main__':
    main(download_many)

8_3 8_5 8_4 8_2 8_1 15_1 13_3 13_2 13_1 

이유는 모르겠으나 에러가 남... 빠르긴 하다.

### 17.1.3 Future는 어디에 있나?

Futre는 concurrent.futures와 asyncio의 내부에 있는 핵심 컴포넌트인데, 이 라이브러의 사용자에게 드러나지 않는 경우가 종종 있다. 여기서는 전반적인 Future의 특징에 대해 설명한다. Future 클래스는 완료되었을 수도 있고 아닐 수도 있는 지연된 계산을 표현하기 위해 사용된다. 대기 중인 작업을 큐에 넣고, 완료 상태를 조사하고, 결과를 가져올 수 있도록 캡슐화 한다.

In [ ]:
# flags_threadpool_ac.py

from concurrent import futures

# flags 모듈의 함수들 재사용
from flags import save_flag, get_flag, show, main

# ThreadPoolExecutor에서 사용할 최대 스레드 수
MAX_WORKERS = 20

# 하나의 이미지를 내려받을 함수
def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.PNG')
    return cc

def download_many(cc_list):
    cc_list= cc_list[:5] # 5개만 쓰자
    # 대기중인 Future 객체를 출력해서 보기 위해 max_worker=3 으로 하드코딩
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            # executor.submit()은 콜러블이 실행되도록 스케줄링하고
            # 이 작업을 나타내는 Future 객체를 반환
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
        
        results = []
        # as_completed()는 Future가 완료해될 때 해당 Future 객체 생성
        for future in futures.as_completed(to_do):
            res = future.result() # 결과 생성
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)

    return len(list(res))

if __name__ == '__main__':
    main(download_many)

Scheduled for 8_1: <Future at 0x7f586bd415c0 state=running>
Scheduled for 8_2: <Future at 0x7f586bd57c18 state=running>
Scheduled for 8_3: <Future at 0x7f586bd57be0 state=running>
Scheduled for 8_4: <Future at 0x7f586bd7f7b8 state=pending>
Scheduled for 8_5: <Future at 0x7f586bd7f860 state=pending>
8_3 <Future at 0x7f586bd57be0 state=finished returned str> result: '8_3'
8_1 <Future at 0x7f586bd415c0 state=finished returned str> result: '8_1'
8_2 <Future at 0x7f586bd57c18 state=finished returned str> result: '8_2'
8_5 <Future at 0x7f586bd7f860 state=finished returned str> result: '8_5'
8_4 

사실 지금까지 테스트한 동시성 스크립트는 어느 것도 병렬로 내려받을 수 없다. concurrent.futures는 전역 인터프리터 락(GIL)에 의해 제한되며, flags_asyncio.py는 단일 스레드로 실행된다. 따라서 다음 질문이 떠오른다.

* 파이썬 스레드가 한 번에 한 스레드만 실행할 수 있게 해주는 GIL에 의해 제한된다면, 어떻게 flags_threadpool.py가 flags.py보다 5배나 빨리 실행될까?
* 둘다 단일 스레드인데, 어떻게 flags_asyncio.py가 flags.py보다 5배나 빨리 실행될 수 있을까?